In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import expm_multiply, expm
from scipy.sparse import diags

from os.path import join, dirname
import sys
sys.path.append(join(".", ".."))
from ionq_circuit_utils import *

import sys
sys.path.append(join(".", "..", ".."))
from utils import *

import json
import hashlib

import networkx as nx
from random import shuffle, seed

from braket.devices import LocalSimulator

from qiskit import QuantumCircuit, transpile

Quantum walk circuit

In [ ]:
def get_qw_circuit(graph, t, r):

    # Returns Trotterized circuit for quantum walk on graph using one-hot embedding

    line_graph = nx.line_graph(graph)
    coloring = nx.coloring.greedy_color(line_graph, strategy="independent_set")

    coloring_grouped = {}
    for edge in coloring.keys():
        if coloring[edge] in coloring_grouped:
            coloring_grouped[coloring[edge]].append(edge)
        else:
            coloring_grouped[coloring[edge]] = [edge]

    num_colors = len(coloring_grouped.keys())

    instructions = []

    # Use randomized first order Trotter
    dt = t / r
    
    np.random.seed(int(t * r))
    for _ in range(r):
        if np.random.rand() < 0.5:
            for color in np.arange(0, num_colors):
                edge_list = coloring_grouped[color]
                
                for i,j in edge_list:
                    instructions.append(get_rxx(dt, targets=[int(i),int(j)]))
                    instructions.append(get_ryy(dt, targets=[int(i),int(j)]))
        else:
            for color in np.arange(0, num_colors)[::-1]:
                edge_list = coloring_grouped[color]
                
                for i,j in edge_list:
                    instructions.append(get_ryy(dt, targets=[int(i),int(j)]))
                    instructions.append(get_rxx(dt, targets=[int(i),int(j)]))
    return instructions

In [ ]:
def run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, task_name, 
           vertex, r, num_shots, device, save_dir, use_real_machine, qpu_job_ids_filename):
    assert encoding == "one-hot"

    job_ids = []
    task_arns = []

    # unnormalized probabilities
    sim_freq = np.zeros((num_time_points, N))
    dimension = 1

    for i, t in enumerate(t_vals):
        
        print(f"Unitless time: {t : 0.3f}")

        # Construct the circuit
        instructions = []

        # Initial state preparation: start from root node
        instructions.append(get_rx(np.pi, vertex))
        if t > 0:
            instructions += get_qw_circuit(graph, t, r)

        if use_real_machine:

            # Create the job json and save it
            job = get_ionq_job_json(task_name, N, dimension, num_shots, device, encoding, instructions, use_native_gates=True)
            
            print(f"Saving in {save_dir}")
            with open(join(save_dir, f"job_{i}.json"), "w") as f:
                json.dump(job, f, default=int)

            # Send the job and get the job id
            job_id = send_job(job)
            print("Job id:", job_id)
            job_ids.append(job_id)

        else:
            
            native_instructions, qubit_phase = get_native_circuit(n, instructions)
            one_qubit_gate_count, two_qubit_gate_count = get_native_gate_counts(native_instructions)
            print(f"1q gates: {one_qubit_gate_count}, 2q gates: {two_qubit_gate_count}")
            circuit = get_braket_native_circuit(native_instructions)
            for j in range(n * dimension):
                circuit.rz(j, -qubit_phase[j] * (2 * np.pi))

            circuit.amplitude(state=bitstrings)

            # Run on simulator
            task = device.run(circuit)

            if device.name == "SV1":
                # Save the data
                metadata = task.metadata()
                task_arn = metadata['quantumTaskArn']
                task_arns.append(task_arn)
                print(f"ARN: {task_arn}")
            else:
                amplitudes = task.result().values[0]
                for j in range(N ** dimension):
                    sim_freq[i,j] = np.abs(amplitudes[bitstrings[j]]) ** 2

    if use_real_machine:
        print(f"Saving IonQ job ids as {qpu_job_ids_filename}")
        with open(join(save_dir, qpu_job_ids_filename), "w") as f:
            json.dump(job_ids, f)
            f.close()
    else:
        return sim_freq

# Quantum walk on 1D chain

In [ ]:
DATA_DIR = "experiment_data"
TASK_DIR = "1d_chain"

CURR_DIR = join("..", "..", "..", DATA_DIR)
check_and_make_dir(CURR_DIR)
CURR_DIR = join(CURR_DIR, TASK_DIR)
check_and_make_dir(CURR_DIR)

print(CURR_DIR)

use_real_machine = False
if use_real_machine:
    device = "qpu.aria-1"
    print("Device:", device)
else:
    device = LocalSimulator()
    print(f"Using {device.name}")

In [ ]:
N = 15
r = 5

dimension = 1
encoding = "one-hot"
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords_1d(n, encoding, periodic=False)
bitstrings = get_bitstrings(N, dimension, encoding)
T = 4.0
num_time_points = 17
t_vals = np.linspace(0, T, num_time_points)

# Trotter steps
num_shots = 200
vertex = int(N/2)

use_error_mitigation = False

if use_error_mitigation:
    assert num_shots >= 500, "Number of shots should be at least 500"

experiment_info = {
    "N": N,
    "dimension": dimension,
    "encoding": encoding,
    "T": T,
    "num_time_points": num_time_points,
    "r": r,
    "num_shots": num_shots,
    "vertex": vertex,
    "use_error_mitigation": use_error_mitigation
}

hash_str = hashlib.md5(json.dumps(experiment_info).encode("utf-8")).hexdigest()
SAVE_DIR = join(CURR_DIR, hash_str)
check_and_make_dir(SAVE_DIR)

print("Save dir:", SAVE_DIR)

with open(join(SAVE_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

graph = nx.path_graph(N)

A = nx.adjacency_matrix(graph, nodelist=sorted(graph.nodes()))

print("Two qubit gate count:", graph.size() * 2 * r)

qpu_job_ids_filename = 'job_ids_qpu.json'

Submit tasks

In [ ]:
if use_real_machine:
    run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)
else:
    sim_freq = run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)

Get tasks

In [ ]:
ionq_freq = get_results(join(SAVE_DIR, qpu_job_ids_filename), num_time_points, codewords, use_error_mitigation)

Post processing and figures

In [ ]:
sim_freq_normalized = np.zeros_like(sim_freq)
for i in range(num_time_points):
    if np.sum(sim_freq[i]) > 0:
        sim_freq_normalized[i] = sim_freq[i] / np.sum(sim_freq[i])

ionq_freq_normalized = np.zeros_like(ionq_freq)
for i in range(num_time_points):
    if np.sum(ionq_freq[i]) > 0:
        ionq_freq_normalized[i] = ionq_freq[i] / np.sum(ionq_freq[i])

num_samples_subspace_ionq = np.sum(ionq_freq, axis=1) * num_shots

In [ ]:
# Ground truth
psi_0 = np.zeros(N)
psi_0[vertex] = 1

psi = expm_multiply(-1j * A, psi_0, start=0, stop=T, num=num_time_points)
ideal_dist = np.abs(psi) ** 2

distance_vec = np.abs(np.arange(-int(N/2), int((N+1)/2)))

# Compute propagation distance
propagation_distance_ideal = ideal_dist @ distance_vec
propagation_distance_sim = sim_freq_normalized @ distance_vec
propagation_distance_ionq = ionq_freq_normalized @ distance_vec

# Using unbiased sample variance to compute standard error
propagation_distance_ionq_err = np.array(
    [np.sqrt(ionq_freq_normalized[i] @ (distance_vec - propagation_distance_ionq[i]) ** 2 / (num_samples_subspace_ionq[i] - 1)) for i in range(num_time_points)]
)

valid_points_ionq = np.sum(ionq_freq, axis=1) > 0

In [ ]:
with open(join(TASK_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()
    
np.savez(join(TASK_DIR, "data.npz"), 
         ideal_dist=ideal_dist,                                         # Ideal QW
         sim_freq=sim_freq,                                             # Simulator with Hamiltonian embedding
         ionq_freq_normalized=ionq_freq_normalized,                     # IonQ (normalized)
         propagation_distance_ideal=propagation_distance_ideal,         # Propagation distance
         propagation_distance_sim=propagation_distance_sim,
         propagation_distance_ionq=propagation_distance_ionq,
         propagation_distance_ionq_err=propagation_distance_ionq_err,   # Propagation distance std error
         ionq_freq=ionq_freq,                                           # IonQ frequency (unnormalized)
         num_samples_subspace_ionq=num_samples_subspace_ionq)           # Samples in encoding subspace

# Quantum walk on 1D cycle

In [ ]:
DATA_DIR = "experiment_data"
TASK_DIR = "1d_cycle"

CURR_DIR = join("..", "..", "..", DATA_DIR)
check_and_make_dir(CURR_DIR)
CURR_DIR = join(CURR_DIR, TASK_DIR)
check_and_make_dir(CURR_DIR)

print(CURR_DIR)

use_real_machine = False
if use_real_machine:
    device = "qpu.aria-1"
    # device = "simulator"
    print("Device:", device)
else:
    device = LocalSimulator()

    print(f"Using {device.name}")

In [ ]:
N = 15
r = 5

dimension = 1
encoding = "one-hot"
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords_1d(n, encoding, periodic=False)
bitstrings = get_bitstrings(N, dimension, encoding)
T = 4.0
num_time_points = 17
t_vals = np.linspace(0, T, num_time_points)

# Trotter steps
num_shots = 200
vertex = int(N/2)

use_error_mitigation = False

if use_error_mitigation:
    assert num_shots >= 500, "Number of shots should be at least 500"

experiment_info = {
    "N": N,
    "dimension": dimension,
    "encoding": encoding,
    "T": T,
    "num_time_points": num_time_points,
    "r": r,
    "num_shots": num_shots,
    "vertex": vertex,
    "use_error_mitigation": use_error_mitigation
}

hash_str = hashlib.md5(json.dumps(experiment_info).encode("utf-8")).hexdigest()
SAVE_DIR = join(CURR_DIR, hash_str)
check_and_make_dir(SAVE_DIR)

print("Save dir:", SAVE_DIR)

with open(join(SAVE_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

graph = nx.cycle_graph(N)

A = nx.adjacency_matrix(graph, nodelist=sorted(graph.nodes()))

print("Two qubit gate count:", graph.size() * 2 * r)

qpu_job_ids_filename = 'job_ids_qpu.json'

In [ ]:
if use_real_machine:
    run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)
else:
    sim_freq = run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)

Get tasks

In [ ]:
ionq_freq = get_results(join(SAVE_DIR, qpu_job_ids_filename), num_time_points, codewords, use_error_mitigation)

Figures and post processing

In [ ]:
sim_freq_normalized = np.zeros_like(sim_freq)
for i in range(num_time_points):
    if np.sum(sim_freq[i]) > 0:
        sim_freq_normalized[i] = sim_freq[i] / np.sum(sim_freq[i])

ionq_freq_normalized = np.zeros_like(ionq_freq)
for i in range(num_time_points):
    if np.sum(ionq_freq[i]) > 0:
        ionq_freq_normalized[i] = ionq_freq[i] / np.sum(ionq_freq[i])

num_samples_subspace_ionq = np.sum(ionq_freq, axis=1) * num_shots

In [ ]:
# Ground truth
psi_0 = np.zeros(N)
psi_0[vertex] = 1

psi = expm_multiply(-1j * A, psi_0, start=0, stop=T, num=num_time_points)
ideal_dist = np.abs(psi) ** 2

distance_vec = np.abs(np.arange(-int(N/2), int((N+1)/2)))

# Compute propagation distance
propagation_distance_ideal = ideal_dist @ distance_vec
propagation_distance_sim = sim_freq_normalized @ distance_vec
propagation_distance_ionq = ionq_freq_normalized @ distance_vec

# Using unbiased sample variance to compute standard error
propagation_distance_ionq_err = np.array(
    [np.sqrt(ionq_freq_normalized[i] @ (distance_vec - propagation_distance_ionq[i]) ** 2 / (num_samples_subspace_ionq[i] - 1)) for i in range(num_time_points)]
)

valid_points_ionq = np.sum(ionq_freq, axis=1) > 0

In [ ]:
with open(join(TASK_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()
    
np.savez(join(TASK_DIR, "data.npz"), 
         ideal_dist=ideal_dist,                                         # Ideal QW
         sim_freq=sim_freq,                                             # Simulator with Hamiltonian embedding
         ionq_freq_normalized=ionq_freq_normalized,                     # IonQ (normalized)
         propagation_distance_ideal=propagation_distance_ideal,         # Propagation distance
         propagation_distance_sim=propagation_distance_sim,
         propagation_distance_ionq=propagation_distance_ionq,
         propagation_distance_ionq_err=propagation_distance_ionq_err,   # Propagation distance std error
         ionq_freq=ionq_freq,                                           # IonQ frequency (unnormalized)
         num_samples_subspace_ionq=num_samples_subspace_ionq)           # Samples in encoding subspace

# Quantum walk on binary tree

In [ ]:
def get_binary_tree(N):
    graph = nx.Graph()
    for i in np.arange(1, N):
        graph.add_edge(int((i-1)/2), int(i))
    return graph

In [ ]:
DATA_DIR = "experiment_data"
TASK_DIR = "binary_tree"

CURR_DIR = join("..", "..", "..", DATA_DIR)
check_and_make_dir(CURR_DIR)
CURR_DIR = join(CURR_DIR, TASK_DIR)
check_and_make_dir(CURR_DIR)

print(CURR_DIR)

use_real_machine = False
if use_real_machine:
    device = "qpu.aria-1"
    print("Device:", device)
else:
    device = LocalSimulator()

    print(f"Using {device.name}")

In [ ]:
N = 15
dimension = 1
encoding = "one-hot"
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords_1d(n, encoding, periodic=False)
bitstrings = get_bitstrings(N, dimension, encoding)
T = 3.0
num_time_points = 16
t_vals = np.linspace(0, T, num_time_points)

# Trotter steps
r = 6
num_shots = 200
vertex = 0

use_error_mitigation = False

if use_error_mitigation:
    assert num_shots >= 500, "Number of shots should be at least 500"

experiment_info = {
    "N": N,
    "dimension": dimension,
    "encoding": encoding,
    "T": T,
    "num_time_points": num_time_points,
    "r": r,
    "num_shots": num_shots,
    "vertex": vertex,
    "use_error_mitigation": use_error_mitigation
}

hash_str = hashlib.md5(json.dumps(experiment_info).encode("utf-8")).hexdigest()
SAVE_DIR = join(CURR_DIR, hash_str)
check_and_make_dir(SAVE_DIR)

print("Save dir:", SAVE_DIR)

with open(join(SAVE_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

    
# Create binary tree with N nodes
graph = get_binary_tree(N)

A = nx.adjacency_matrix(graph, nodelist=sorted(graph.nodes()))

print("Two qubit gate count:", graph.size() * 2 * r)

qpu_job_ids_filename = "job_ids_qpu.json"

Submit tasks

In [ ]:
if use_real_machine:
    run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)
else:
    sim_freq = run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)

In [ ]:
ionq_freq = get_results(join(SAVE_DIR, qpu_job_ids_filename), num_time_points, codewords, use_error_mitigation)

Post processing and figures

In [ ]:
sim_freq_normalized = np.zeros_like(sim_freq)
for i in range(num_time_points):
    if np.sum(sim_freq[i]) > 0:
        sim_freq_normalized[i] = sim_freq[i] / np.sum(sim_freq[i])

ionq_freq_normalized = np.zeros_like(ionq_freq)
for i in range(num_time_points):
    if np.sum(ionq_freq[i]) > 0:
        ionq_freq_normalized[i] = ionq_freq[i] / np.sum(ionq_freq[i])

num_samples_subspace_ionq = np.sum(ionq_freq, axis=1) * num_shots

In [ ]:
# Ground truth
psi_0 = np.zeros(N)
psi_0[vertex] = 1

psi = expm_multiply(-1j * A, psi_0, start=0, stop=T, num=num_time_points)
ideal_dist = np.abs(psi) ** 2

distance_vec = np.zeros(N)
for i in range(N):
    distance_vec[i] = int(np.log2(i + 1))

# Compute propagation distance
propagation_distance_ideal = ideal_dist @ distance_vec
propagation_distance_sim = sim_freq_normalized @ distance_vec
propagation_distance_ionq = ionq_freq_normalized @ distance_vec

# Using unbiased sample variance
propagation_distance_ionq_err = np.array(
    [np.sqrt(ionq_freq_normalized[i] @ (distance_vec - propagation_distance_ionq[i]) ** 2 / (num_samples_subspace_ionq[i] - 1)) for i in range(num_time_points)]
)

valid_points_ionq = np.sum(ionq_freq, axis=1) > 0

In [ ]:
with open(join(TASK_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()
    
np.savez(join(TASK_DIR, "data.npz"), 
         ideal_dist=ideal_dist,                                         # Ideal QW
         sim_freq=sim_freq,                                             # Simulator with Hamiltonian embedding
         ionq_freq_normalized=ionq_freq_normalized,                     # IonQ (normalized)
         propagation_distance_ideal=propagation_distance_ideal,         # Propagation distance
         propagation_distance_sim=propagation_distance_sim,
         propagation_distance_ionq=propagation_distance_ionq,
         propagation_distance_ionq_err=propagation_distance_ionq_err,   # Propagation distance std error
         ionq_freq=ionq_freq,                                           # IonQ frequency (unnormalized)
         num_samples_subspace_ionq=num_samples_subspace_ionq)           # Samples in encoding subspace

# Quantum walk on glued trees graph

In [ ]:
DATA_DIR = "experiment_data"
TASK_DIR = "glued_trees"

CURR_DIR = join("..", "..", "..", DATA_DIR)
check_and_make_dir(CURR_DIR)
CURR_DIR = join(CURR_DIR, TASK_DIR)
check_and_make_dir(CURR_DIR)

print(CURR_DIR)

use_real_machine = False
if use_real_machine:
    device = "qpu.aria-1"
    print("Device:", device)
else:
    device = LocalSimulator()
    print(f"Using {device.name}")

In [ ]:
def get_glued_tree(h):
    seed(0)
    # Two binary trees of height h (2^(h+1) - 1 nodes each glued together
    num_nodes_per_binary_tree = 2 ** (h+1) - 1
    num_nodes = 2 * num_nodes_per_binary_tree
    graph = nx.Graph()

    # Leaves
    leaves_first = []
    leaves_second = []
    for i in range(2 ** h):
        leaves_first.append(2 ** h - 1 + i)
        leaves_second.append(num_nodes - 1 - (2 ** h - 1) - i)

    for i in np.arange(1, num_nodes_per_binary_tree):

        # First binary tree
        graph.add_edge(int((i-1)/2), int(i))

        # Second binary tree
        graph.add_edge(int(num_nodes - 1 - int((i-1)/2)), int(num_nodes - 1 - i))

    # Glue the two trees together
    # Shuffle the leaves to get a random cycle
    shuffle(leaves_first)
    shuffle(leaves_second)

    for i in range(2 ** h):
        graph.add_edge(int(leaves_first[i]), int(leaves_second[i]))
        graph.add_edge(int(leaves_second[i]), int(leaves_first[(i+1) % (2 ** h)]))

    return graph

In [ ]:
glued_tree_height = 2
N = 2 * (2 ** (glued_tree_height + 1) - 1)
print(f"Nodes in graph: {N}")

dimension = 1
encoding = "one-hot"
n = num_qubits_per_dim(N, encoding=encoding)
codewords = get_codewords_1d(n, encoding, periodic=False)
bitstrings = get_bitstrings(N, dimension, encoding)

T = 2
r = 4

num_time_points = 11
t_vals = np.linspace(0, T, num_time_points)

num_shots = 200
vertex = 0

use_error_mitigation = False

if use_error_mitigation:
    assert num_shots >= 500, "Number of shots should be at least 500"

experiment_info = {
    "N": N,
    "dimension": dimension,
    "encoding": encoding,
    "T": T,
    "num_time_points": num_time_points,
    "r": r,
    "num_shots": num_shots,
    "vertex": vertex,
    "use_error_mitigation": use_error_mitigation,
}

hash_str = hashlib.md5(json.dumps(experiment_info).encode("utf-8")).hexdigest()
SAVE_DIR = join(CURR_DIR, hash_str)
check_and_make_dir(SAVE_DIR)
print("Save dir:", SAVE_DIR)

with open(join(SAVE_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()
    
graph = get_glued_tree(glued_tree_height)

nx.draw(graph, with_labels=True)
A = nx.adjacency_matrix(graph, nodelist=sorted(graph.nodes()))

print("Two qubit gate count:", graph.size() * r * 2)

qpu_job_ids_filename = 'job_ids_qpu.json'

Submit tasks

In [ ]:
if use_real_machine:
    run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)
else:
    sim_freq = run_qw(N, dimension, n, encoding, bitstrings, num_time_points, graph, t_vals, TASK_DIR, vertex, r, num_shots, device, SAVE_DIR, use_real_machine, qpu_job_ids_filename)

Get data from completed tasks

In [ ]:
ionq_freq = get_results(join(SAVE_DIR, qpu_job_ids_filename), num_time_points, codewords, use_error_mitigation)

Post processing and figures

In [ ]:
ionq_freq_normalized = np.zeros_like(ionq_freq)
for i in range(num_time_points):
    if np.sum(ionq_freq[i]) > 0:
        ionq_freq_normalized[i] = ionq_freq[i] / np.sum(ionq_freq[i])

num_samples_subspace_ionq = np.sum(ionq_freq, axis=1) * num_shots

In [ ]:
# Ground truth
psi_0 = np.zeros(N)
psi_0[vertex] = 1

psi = expm_multiply(-1j * A, psi_0, start=0, stop=T, num=num_time_points)
ideal_dist = np.abs(psi) ** 2

def get_glued_tree_distance_vec(glued_tree_height):
    
    distance_vec = np.zeros(N)
    for i in range(2 ** (glued_tree_height + 1) - 1):
        distance_vec[i] = int(np.log2(i + 1))
    for i in np.arange(2 ** (glued_tree_height + 1) - 1, N):
        distance_vec[i] = 2 * (glued_tree_height + 1) - 1 - int(np.log2(N - i))

    return distance_vec

In [ ]:
with open(join(TASK_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

np.savez(join(TASK_DIR, "data.npz"), 
         ideal_dist=ideal_dist,
         sim_freq=sim_freq,
         ionq_freq=ionq_freq,
         ionq_freq_normalized=ionq_freq_normalized,
         num_samples_subspace_ionq=num_samples_subspace_ionq)